In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, udf
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.appName('Spark DF').getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True) 
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 10)
fp = '../FileStore/tables/OfficeData.csv'
df = spark.read.options(header=True, inferSchema=True).csv(fp)

In [46]:
@udf
def calc_increment(state, salary, bonus):
    def increment(x, y):
        return salary * x + bonus * y
    return increment(0.1, 0.05) if state == 'NY' else increment(0.12, 0.03)

df.withColumn('increment', calc_increment(col('state'), col('salary'), col('bonus')))

employee_name,department,state,salary,age,bonus,increment
James,Sales,NY,90000,34,10000,9500.0
Michael,Sales,NY,86000,56,20000,9600.0
Robert,Sales,CA,81000,30,23000,10410.0
Maria,Finance,CA,90000,24,23000,11490.0
Raman,Finance,CA,99000,40,24000,12600.0
Scott,Finance,NY,83000,36,19000,9250.0
Jen,Finance,NY,79000,53,15000,8650.0
Jeff,Marketing,CA,80000,25,18000,10140.0
Kumar,Marketing,NY,91000,50,21000,10150.0


In [48]:
increment_udf = udf(lambda st, sal, bon: (sal * 0.1 + bon * 0.05) if st == 'NY' else (sal * 0.12 + bon * 0.03))

df.withColumn('increment', increment_udf(col('state'), col('salary'), col('bonus')))

employee_name,department,state,salary,age,bonus,increment
James,Sales,NY,90000,34,10000,9500.0
Michael,Sales,NY,86000,56,20000,9600.0
Robert,Sales,CA,81000,30,23000,10410.0
Maria,Finance,CA,90000,24,23000,11490.0
Raman,Finance,CA,99000,40,24000,12600.0
Scott,Finance,NY,83000,36,19000,9250.0
Jen,Finance,NY,79000,53,15000,8650.0
Jeff,Marketing,CA,80000,25,18000,10140.0
Kumar,Marketing,NY,91000,50,21000,10150.0


employee_name,department,state,salary,age,bonus,increment
James,Sales,NY,90000,34,10000,9500.0
Michael,Sales,NY,86000,56,20000,9600.0
Robert,Sales,CA,81000,30,23000,10410.0
Maria,Finance,CA,90000,24,23000,11490.0
Raman,Finance,CA,99000,40,24000,12600.0
Scott,Finance,NY,83000,36,19000,9250.0
Jen,Finance,NY,79000,53,15000,8650.0
Jeff,Marketing,CA,80000,25,18000,10140.0
Kumar,Marketing,NY,91000,50,21000,10150.0
